In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import datetime
import string
from string import digits
import collections
import scipy.stats as scs
import cc_pipeline as P
import time
import random
import pickle
from pprint import pprint
from collections import Counter

#sentiment and language
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
import vaderSentiment
from langdetect import detect
from gensim.models import Word2Vec
from gensim import corpora
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from spacy import displacy

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import Pipeline
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF, LatentDirichletAllocation
from gensim.models.ldamodel import LdaModel
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import chi2
import knee_locator

#plotting
from bokeh.plotting import figure, show, output_file, output_notebook, ColumnDataSource
from bokeh.models import HoverTool, BoxSelectTool
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import pyLDAvis.sklearn
import pyLDAvis.gensim as gensimvis
import pyLDAvis
import umap

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [63]:
def inline_text(show_raw):
    
    '''returns show text without timestamps'''
    
    temp = " ".join( ["\n".join( x.split("\n")[2:] ) for x in show_raw.split("\n\n")] )
    temp = temp.split('\n')
    temp = " ".join(temp)
    return temp

def clean_for_spacy(text_list):
    
    '''cleans all text and creates new column in dataframe'''
    
    doc_list = []
    for word in text_list:
        doc_list.append(inline_text(word))
    return doc_list

def clean_for_spacy_lower(text_list):
    
    '''cleans all text and creates new column in dataframe'''
    
    doc_list = []
    for word in text_list:
        doc_list.append(inline_text(word).lower())
    return doc_list

def lang_detect(doc_series):
    
    lang = []
    for x in doc_series:
        eng = 'en'
        span = 'es'

        try:
            if detect(x) == eng:
                lang.append(eng)
            else:
                lang.append(span)
        except:
            lang.append(None)
            
    return lang

def get_orgs(chunks):
    
    nlp = spacy.load('en_core_web_sm')
    orgs = []
    
    for chunk in chunks:
        document = nlp(chunk)
        labels = set([w.label_ for w in document.ents]) 

        for label in labels: 

            temp_entities = [e for e in document.ents if label==e.label_] 
            temp_entities = list(set(temp_entities)) 

            if label == 'ORG':
                orgs.append(str(temp_entities))
                
    orgs = " ".join(orgs)
    return orgs

In [44]:
sent_df = pd.read_csv('data/cc_1000_text.csv', encoding='utf=8')

In [45]:
#english only (for testing)

doc_series = pd.Series(sent_df['text'].values)
language = lang_detect(doc_series)
sent_df['language'] = language
english = sent_df[sent_df['language'] == 'en']

In [46]:
temp_text = english['text'].values
sent_text = clean_for_spacy(temp_text)
sent_text_lower = clean_for_spacy_lower(temp_text)

In [47]:
corpus = " ".join(sent_text)

In [48]:
#break into chunks for spaCy

n = 100000
chunks = [corpus[i:i+n] for i in range(0, len(corpus), n)]
len(chunks)

121

In [49]:
org_list = get_orgs(chunks)
org_list

'[TREAT, LAUGH, \xa0, Lincoln, THE LIFE \xa0OF A, OUT, NOW, KIND OF TYPICAL, BIG ON, SEC, Thompson, Allstate, TIDE, GIANT, EACH NIGHT, LAUGHS, MOMENT, END, CHICLET, FORM, FINALLY, \xa0, \xa0, \xa0, \xa0, \xa0, \xa0, T. Rowe Price, KNOW, Lincoln, VE, VE GOT, HIGHEST, KID, TYPICALLY, GOT, BASICALLY, TOP, \xa0, \xa0, \xa0, ESPERANZA, RACING STABLE, Medicare, \xa0MILITARY’S, LOWS, GARY HARRIS ROAD GAME AT, HAPPEN, SHY OF, LOT OF WAYS, INEZ, Jeep, OPENED FIRE ON, \xa0, KILLING, CAMERA, Carnegie Hall, GAME A TRAP GAME, \xa0, CAR, BACK NOW, FATALITY, THE DEFINING MOMENT \xa0, HONOR, CLAP, TRU, COURSE, ESPECIALLY, LET, \xa0, KOCH, THE COLUMBIA RIVER GORGE, SEE, SET, SEC, TYPICALLY, SANTA’S, Philadelphia Eagles, KNOW, LL, VE, BRAIN, \xa0, VIDEO, Deb:, DRIFTER, DO, Deb: BOB HEYE, LOCAL TIME, COMPLAINED OF A SEVERE \xa0WATER, @DEBKNAPPKATU, \xa0, \xa0, CDC, COVER, GAME, FEATURE \xa0, SANTA ONCE BROUGHT ME, LAUGHS, ELFIE, VE, PLENTY OF AIR, WEVE RAISED, DO, Farmers, McDonald, Sleep, YOURS OR MINE,

In [83]:
with open("brand_list.txt") as f:
    lines = f.read().replace('\n', '').lower()
print(lines)   

['accenture', 'adidas', 'adobe', 'agricultural bank of china', 'aia', 'aldi', 'alibaba', 'amazon', 'american express', 'anz', 'apple', 'at&t', 'baidu', 'bank of america', 'bank of china', 'bca', 'bmw', 'bt', 'budweiser', 'chase', 'china construction bank', 'china life', 'china mobile', 'cisco', 'citi', 'coca-cola', 'colgate', 'commonwealth bank of australia', 'costco', 'deutsche telekom', 'dhl', 'disney', 'ebay', 'exxonmobil', 'facebook', 'fedex', 'ford', 'ge', 'gillette', 'google', 'gucci', 'hdfc bank', 'hermès', 'honda', 'hp', 'hsbc', 'huawei', 'ibm', 'icbc', 'ikea', 'instagram', 'intel', 'j.p. morgan', 'jd.com', 'kfc', "l'oréal paris", 'linkedin', 'louis vuitton', "lowe's", 'marlboro', 'mastercard', "mcdonald's", 'mercedes-benz', 'microsoft', 'moutai', 'movistar', 'netflix', 'nike', 'ntt', 'oracle', 'orange', 'pampers', 'paypal', 'pepsi', 'ping an', 'rbc', 'salesforce', 'samsung', 'sap', 'sf express', 'shell', 'siemens', 'spectrum', 'starbucks', 'subway', 'td', 'tencent', 'the home 

In [84]:
# with open ('data/brands.pkl', 'wb') as f:
#     pickle.dump(lines, f)
    
# with open ('data/brands.pkl', 'rb') as r:
#     brands = pickle.load(r)

In [ ]:
#get sentiment if brand present in text

def get_sentiment_score(doc, brands):
    
    if any(brand in doc for brand in brands):
        
        doc =spacy sentiment








